In [1]:
#Connect to google drive to access the files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#library imports
  #required to run the ML and the preprocessing stages, plus visualization
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import seaborn as sns
import os
from matplotlib.pyplot import figure
from collections import Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score
from keras import Sequential
from keras import optimizers
from keras.optimizers import SGD



In [10]:
df_data = pd.read_csv('/content/drive/MyDrive/ESOF-4969 Degree Project/Machine Learning/Production_Data_final.csv')#read the dataset

In [4]:
df_data.head()

,Unnamed: 0,TrackID,TrackName,Album,PreviewURL,ArtistName,Genre,Duration,Danceability,Energy,...,Channel2_318,Channel2_319,Channel2_320,Channel2_321,Channel2_322,Channel2_323,Channel2_324,Channel2_325,Channel2_326,Channel2_327
0,0,37Jdi2cCSRA5jl6zADeLOy,D R O V E MY CAR,1 Trait High,https://p.scdn.co/mp3-preview/ab54f8f789175d42...,1 Trait Danger,comedy rap,120937,0.601,0.827,...,896,-18226,-5812,4875,-7218,8262,14948,-10,8951,-6552
1,1,14jLrcO0NGIxtX56LHEhsv,Reprisal,1 Trait World Tour,https://p.scdn.co/mp3-preview/ffe73947f36e6e15...,1 Trait Danger,comedy rap,100634,0.391,0.575,...,-1289,2746,1653,614,1316,-472,3823,672,-251,2903
2,2,7dcNDXD9SwJ4Txqg2fE2t8,Danger,Back from the Dead,https://p.scdn.co/mp3-preview/1ae38bb85e0313f6...,LUCKY3RD,houston rap,181395,0.802,0.511,...,-1197,-1213,-3706,-1938,-3796,361,-140,1618,-494,-1781
3,3,0ackjn8zBcLQnkeRPeOCVj,Danger,Danger,https://p.scdn.co/mp3-preview/ab856e3e92deddc1...,Cooliecut,dark trap,96052,0.720,0.569,...,3429,9928,-26812,10141,17013,16055,16658,12518,19679,8613
4,4,1wDY9VWdCoRjLV4gUyYf52,Danger,In Transit (Deluxe),https://p.scdn.co/mp3-preview/61586296e9713d07...,Terri,nigerian pop,157648,0.514,0.582,...,-220,-6064,9413,4410,-177,507,6667,-8546,-20569,2043


In [11]:
df_data_better = df_data.drop(['Unnamed: 0'],axis=1) #Drops the unnamed column which is added when combining csv

In [12]:
df_trimmed = df_data_better.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre','Duration'], axis=1)
df_trimmed
#training data

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,...,Channel2_318,Channel2_319,Channel2_320,Channel2_321,Channel2_322,Channel2_323,Channel2_324,Channel2_325,Channel2_326,Channel2_327
0,0.601,0.827,2,-7.056,0.1350,0.01180,0.000000,0.9610,0.2930,128.055,...,896,-18226,-5812,4875,-7218,8262,14948,-10,8951,-6552
1,0.391,0.575,5,-7.520,0.1070,0.38000,0.004660,0.4130,0.3260,143.260,...,-1289,2746,1653,614,1316,-472,3823,672,-251,2903
2,0.802,0.511,6,-9.651,0.2980,0.03120,0.000000,0.0843,0.3320,84.979,...,-1197,-1213,-3706,-1938,-3796,361,-140,1618,-494,-1781
3,0.720,0.569,5,-6.983,0.0607,0.01650,0.000525,0.5400,0.1690,119.989,...,3429,9928,-26812,10141,17013,16055,16658,12518,19679,8613
4,0.514,0.582,11,-5.243,0.3880,0.35500,0.000001,0.1840,0.4530,66.681,...,-220,-6064,9413,4410,-177,507,6667,-8546,-20569,2043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,0.316,0.110,9,-17.251,0.0367,0.99200,0.539000,0.4030,0.1220,88.791,...,-1421,-745,3733,5421,541,463,-1944,1936,-11179,-4804
71147,0.355,0.362,1,-13.699,0.0340,0.70900,0.623000,0.1220,0.3570,86.946,...,776,-2015,1290,1881,-5497,-811,-4471,120,3245,8489
71148,0.591,0.973,1,-2.835,0.0471,0.00420,0.390000,0.1850,0.0419,125.974,...,-19519,-13232,-8683,-745,-2270,10022,661,-12666,2892,-6850
71149,0.522,0.988,1,-4.646,0.3210,0.00320,0.051300,0.1200,0.0689,141.685,...,7484,3477,6979,-7515,-14778,12483,12314,-1127,7092,-2415


In [7]:
#df_trimmed.iloc[:,677] #first channel col they go from 24 - 677   so encode 24 - 238, 239 - 478, 479 - 677

In [21]:
df_autoencode_col1 = df_trimmed.iloc[:,24:238] 
df_autoencode_col1

,Channel1_1,Channel1_2,Channel1_3,Channel1_4,Channel1_5,Channel1_6,Channel1_7,Channel1_8,Channel1_9,Channel1_10,...,Channel1_205,Channel1_206,Channel1_207,Channel1_208,Channel1_209,Channel1_210,Channel1_211,Channel1_212,Channel1_213,Channel1_214
0,-4939,4194,-17352,-11871,7825,-3263,415,14806,11231,-9412,...,-11733,2678,8387,5506,3732,12513,-7228,22984,5390,-1772
1,-2246,11016,3776,-3226,-3583,-2998,302,3774,1259,10551,...,-3657,10186,1503,13885,7763,5730,12272,1130,-1511,-3637
2,-3223,-12447,-9906,15064,14835,-9678,-7472,-21299,-2118,-406,...,3471,1735,3138,5581,6563,-929,-586,-1014,2864,-2069
3,-13728,-17200,-25698,-20324,-15013,-6440,-838,1555,2313,3070,...,-13610,-13448,-20283,-17177,-24091,-18966,2666,14508,15417,14645
4,6464,7954,-1882,14061,29068,-2079,6771,-3701,-8761,4913,...,7837,5888,-18432,5419,22291,-19052,-4096,24170,-3508,-12830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,4662,-511,60,3401,-1365,1575,-2072,-2090,2602,1726,...,555,-308,-1245,-423,-1170,657,704,138,175,-531
71147,-5918,2886,8858,7088,1158,2018,205,3419,-1652,1861,...,-11204,-1157,-1099,-4230,9323,6413,-5064,-600,-4319,-344
71148,5313,-583,-3981,-6023,-2219,4271,2331,3638,496,-5468,...,-8999,3184,20605,-11299,8704,27786,-21904,-7278,5154,8945
71149,-7473,-1399,17234,18489,2763,12567,8296,9652,-624,-5688,...,-3084,-4161,5835,-10751,-12252,-1748,-9073,-3636,-2501,6878


In [24]:
df_autoencode_col2 = df_trimmed.iloc[:,238:458] 
df_autoencode_col2

,Channel1_215,Channel1_216,Channel1_217,Channel1_218,Channel1_219,Channel1_220,Channel1_221,Channel1_222,Channel1_223,Channel1_224,...,Channel2_98,Channel2_99,Channel2_100,Channel2_101,Channel2_102,Channel2_103,Channel2_104,Channel2_105,Channel2_106,Channel2_107
0,3688,17333,-7104,9718,12091,8191,-7243,18101,-3726,2681,...,9852,-13160,12145,16151,10696,-18477,12583,-6585,6214,2670
1,543,3534,-2267,2661,-7165,-18708,-14092,-17268,-12652,-132,...,-4624,-10116,-13450,-6970,-12398,-17286,-8346,-16259,-69,501
2,-5040,7644,1865,-718,-12854,17857,-21657,8285,-1187,4825,...,-2807,-1657,7950,16212,-11512,3245,7834,-19571,13773,-10901
3,2162,19838,12908,13220,14433,10498,9972,22515,12238,19649,...,4754,-3005,9848,-18386,-16777,-13692,-14870,-9948,-9557,-11671
4,12929,22429,-12469,3131,20567,-4343,5313,-3357,2023,-3168,...,-2163,-19183,9031,4075,-24119,1962,14064,-17832,-1985,6982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,444,1135,-177,-5747,104,1661,-4222,-2963,2467,466,...,164,695,7628,-480,-986,-3705,6255,3693,7504,6166
71147,-235,-12410,-6436,1540,-8401,-248,4600,-2976,-247,-3432,...,647,-6495,3410,8522,933,1293,-555,-1310,-918,-5172
71148,-5676,-167,-3853,-1153,1811,10113,15844,-782,10565,7230,...,-19748,7600,21309,-362,2509,24638,3922,16662,-27042,4234
71149,26,-6392,5310,-2978,2000,500,-7743,-1773,14624,-19716,...,9442,-8481,-15540,-694,1631,7676,-12455,20038,6304,1117


In [25]:
df_autoencode_col3 = df_trimmed.iloc[:,458:678] 
df_autoencode_col3

,Channel2_108,Channel2_109,Channel2_110,Channel2_111,Channel2_112,Channel2_113,Channel2_114,Channel2_115,Channel2_116,Channel2_117,...,Channel2_318,Channel2_319,Channel2_320,Channel2_321,Channel2_322,Channel2_323,Channel2_324,Channel2_325,Channel2_326,Channel2_327
0,10492,1507,5264,13494,-12382,-142,19221,3157,7951,13971,...,896,-18226,-5812,4875,-7218,8262,14948,-10,8951,-6552
1,4125,8630,6377,11607,4337,4258,17227,-185,7723,5761,...,-1289,2746,1653,614,1316,-472,3823,672,-251,2903
2,13753,-11993,935,13271,-11406,-390,11122,400,926,9413,...,-1197,-1213,-3706,-1938,-3796,361,-140,1618,-494,-1781
3,4301,2385,8580,-10836,-10780,-6050,-21256,-23524,-15624,4939,...,3429,9928,-26812,10141,17013,16055,16658,12518,19679,8613
4,-2307,-16732,7752,3314,-20300,17859,394,1011,-17488,5885,...,-220,-6064,9413,4410,-177,507,6667,-8546,-20569,2043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,-7918,4202,-4206,978,9505,-3282,5599,-6297,7025,1100,...,-1421,-745,3733,5421,541,463,-1944,1936,-11179,-4804
71147,4537,-11203,-4601,2595,-206,1539,-2105,3098,-7465,3930,...,776,-2015,1290,1881,-5497,-811,-4471,120,3245,8489
71148,-12722,-13067,26263,14152,11096,23214,-6168,16421,-3986,14406,...,-19519,-13232,-8683,-745,-2270,10022,661,-12666,2892,-6850
71149,9811,-13090,783,6220,13331,6745,18,4986,2501,9575,...,7484,3477,6979,-7515,-14778,12483,12314,-1127,7092,-2415


NeuralNet stuff

sklearn SOM

In [11]:
pip install sklearn-som


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
from sklearn_som.som import SOM

minmaxscaling all dfs


In [13]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range= (0,1))
df_trimmed=sc.fit_transform(df_trimmed)
df_trimmed

array([[0.60953347, 0.827     , 0.18181818, ..., 0.49985504, 0.63659113,
        0.40003052],
       [0.39655172, 0.575     , 0.45454545, ..., 0.51026169, 0.49617762,
        0.54430457],
       [0.81338742, 0.511     , 0.54545455, ..., 0.52469673, 0.49246967,
        0.47283131],
       ...,
       [0.59939148, 0.973     , 0.09090909, ..., 0.30673686, 0.54413672,
        0.39548333],
       [0.52941176, 0.988     , 0.09090909, ..., 0.48281071, 0.60822461,
        0.46315709],
       [0.94827586, 0.484     , 0.45454545, ..., 0.52785534, 0.49982452,
        0.49941253]])

In [14]:

df_autoencode_col1=sc.fit_transform(df_autoencode_col1)
df_autoencode_col1

array([[0.42464332, 0.56400397, 0.23523308, ..., 0.5569543 , 0.69094377,
        0.38971542],
       [0.46573587, 0.66810101, 0.5576257 , ..., 0.61846342, 0.58744182,
        0.68726635],
       [0.4508278 , 0.31007858, 0.34885176, ..., 0.60015259, 0.485832  ,
        0.49106584],
       ...,
       [0.58107881, 0.49111162, 0.43926146, ..., 0.63282216, 0.92399481,
        0.16577401],
       [0.38597696, 0.47866026, 0.76298161, ..., 0.31305409, 0.47333486,
        0.36156252],
       [0.61886015, 0.55141527, 0.48647288, ..., 0.56237125, 0.17326619,
        0.28801404]])

In [15]:

df_autoencode_col2=sc.fit_transform(df_autoencode_col2)
df_autoencode_col2

array([[0.58225376, 0.47296864, 0.5562829 , ..., 0.21806668, 0.6920119 ,
        0.39952697],
       [0.47695125, 0.44451057, 0.50829328, ..., 0.23624018, 0.37265583,
        0.25191119],
       [0.54370947, 0.46843671, 0.42310216, ..., 0.54952316, 0.61954681,
        0.20137331],
       ...,
       [0.57865263, 0.63649958, 0.41339742, ..., 0.87595941, 0.55985351,
        0.75425345],
       [0.46184482, 0.60495918, 0.50040436, ..., 0.61713588, 0.30995651,
        0.80576791],
       [0.56450752, 0.49864958, 0.50008392, ..., 0.50084688, 0.49909209,
        0.29359884]])

In [16]:

df_autoencode_col3=sc.fit_transform(df_autoencode_col3)
df_autoencode_col3

array([[0.54074922, 0.66010529, 0.52300298, ..., 0.72809949, 0.49985504,
        0.63659113],
       [0.5076524 , 0.56295109, 0.63169299, ..., 0.55834287, 0.51026169,
        0.49617762],
       [0.33366903, 0.70986496, 0.31700618, ..., 0.49787137, 0.52469673,
        0.49246967],
       ...,
       [0.56461433, 0.30588235, 0.30061799, ..., 0.51009384, 0.30673686,
        0.54413672],
       [0.51705196, 0.64971389, 0.30026703, ..., 0.68790723, 0.48281071,
        0.60822461],
       [0.52034791, 0.49254597, 0.53904021, ..., 0.49683375, 0.52785534,
        0.49982452]])

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LeakyReLU

In [18]:
input1 = Input(shape=df_autoencode_col1.shape[1:]) #gets the size of the number of features in the df i am pulling from
encoder = Dense(128)(input1) 
encoder = LeakyReLU(alpha=0.15)(encoder)#leakyrelu used in hidden layers
encoder = Dense(64)(encoder)
encoder = LeakyReLU(alpha=0.15)(encoder)
# latent space with tanh actiavtion
latent_space1 = Dense(22, activation="tanh")(encoder) #bottleneck layer, used

decoder = Dense(64)(latent_space1)
decoder = LeakyReLU(alpha=0.15)(decoder)
decoder = Dense(128)(decoder)
decoder = LeakyReLU(alpha=0.15)(decoder)
decoder = Dense(units=df_autoencode_col1.shape[1], activation="tanh")(decoder) #never used as pulling from latent space

autoencoder = Model(input1, decoder)
autoencoder.compile(optimizer = "adam", metrics=["mse"], loss="mse")#compiling model with the neccesary optimizer and loss
autoencoder.fit(df_autoencode_col1, df_autoencode_col1, epochs=12, batch_size=25, validation_split=0.2)#fits the autoencoder, 12 epoch, 25 batch size

Epoch 1/12
2277/2277 [==============================] - 24s 7ms/step - loss: 0.0194 - mse: 0.0194 - val_loss: 0.0123 - val_mse: 0.0123
Epoch 2/12
2277/2277 [==============================] - 11s 5ms/step - loss: 0.0173 - mse: 0.0173 - val_loss: 0.0119 - val_mse: 0.0119
Epoch 3/12
2277/2277 [==============================] - 11s 5ms/step - loss: 0.0169 - mse: 0.0169 - val_loss: 0.0117 - val_mse: 0.0117
Epoch 4/12
2277/2277 [==============================] - 9s 4ms/step - loss: 0.0168 - mse: 0.0168 - val_loss: 0.0117 - val_mse: 0.0117
Epoch 5/12
2277/2277 [==============================] - 11s 5ms/step - loss: 0.0167 - mse: 0.0167 - val_loss: 0.0117 - val_mse: 0.0117
Epoch 6/12
2277/2277 [==============================] - 11s 5ms/step - loss: 0.0167 - mse: 0.0167 - val_loss: 0.0116 - val_mse: 0.0116
Epoch 7/12
2277/2277 [==============================] - 11s 5ms/step - loss: 0.0166 - mse: 0.0166 - val_loss: 0.0116 - val_mse: 0.0116
Epoch 8/12
2277/2277 [==============================] - 

In [19]:
encoder1 = Model(input1, latent_space1)

In [20]:
test_au_features1 = encoder1.predict(df_autoencode_col1)
test_au_features1 #will be used for the final autoencoder to make the 16 attribute encoder

2224/2224 [==============================] - 3s 1ms/step


array([[ 0.3250402 ,  0.46454063, -0.21095058, ...,  0.22671442,
         0.10142745, -0.13477114],
       [ 0.3409306 ,  0.23374566, -0.05517427, ...,  0.10855085,
         0.06269536,  0.16520648],
       [ 0.2456839 ,  0.37355044, -0.11077284, ...,  0.11760463,
         0.16912901, -0.03442059],
       ...,
       [ 0.18246998,  0.34141067,  0.20425063, ...,  0.24088827,
         0.05409478, -0.23445521],
       [ 0.18386932,  0.07029153,  0.00934042, ..., -0.03494829,
        -0.06076352, -0.10688417],
       [ 0.19028294,  0.2641175 ,  0.22949432, ..., -0.0291658 ,
        -0.02373205, -0.21428436]], dtype=float32)

encoder #2

In [21]:
input2 = Input(shape=df_autoencode_col2.shape[1:]) #same as last encoder but differnt dataframe
encoder = Dense(128)(input2)
encoder = LeakyReLU(alpha=0.15)(encoder)
encoder = Dense(64)(encoder)
encoder = LeakyReLU(alpha=0.15)(encoder)

latent_space2 = Dense(22, activation="tanh")(encoder)

decoder = Dense(64)(latent_space2)
decoder = LeakyReLU(alpha=0.15)(decoder)
decoder = Dense(128)(decoder)
decoder = LeakyReLU(alpha=0.15)(decoder)
decoder = Dense(units=df_autoencode_col2.shape[1], activation="tanh")(decoder)

autoencoder = Model(input2, decoder)
autoencoder.compile(optimizer = "adam",metrics=["mse"],loss="mse")
autoencoder.fit(df_autoencode_col2, df_autoencode_col2, epochs=12, batch_size=25, validation_split=0.2)

Epoch 1/12
2277/2277 [==============================] - 21s 8ms/step - loss: 0.0184 - mse: 0.0184 - val_loss: 0.0116 - val_mse: 0.0116
Epoch 2/12
2277/2277 [==============================] - 11s 5ms/step - loss: 0.0164 - mse: 0.0164 - val_loss: 0.0112 - val_mse: 0.0112
Epoch 3/12
2277/2277 [==============================] - 11s 5ms/step - loss: 0.0161 - mse: 0.0161 - val_loss: 0.0112 - val_mse: 0.0112
Epoch 4/12
2277/2277 [==============================] - 9s 4ms/step - loss: 0.0160 - mse: 0.0160 - val_loss: 0.0111 - val_mse: 0.0111
Epoch 5/12
2277/2277 [==============================] - 10s 4ms/step - loss: 0.0160 - mse: 0.0160 - val_loss: 0.0111 - val_mse: 0.0111
Epoch 6/12
2277/2277 [==============================] - 10s 4ms/step - loss: 0.0160 - mse: 0.0160 - val_loss: 0.0111 - val_mse: 0.0111
Epoch 7/12
2277/2277 [==============================] - 11s 5ms/step - loss: 0.0159 - mse: 0.0159 - val_loss: 0.0111 - val_mse: 0.0111
Epoch 8/12
2277/2277 [==============================] - 

In [22]:
encoder2 = Model(input2, latent_space2)

In [23]:
test_au_features2 = encoder2.predict(df_autoencode_col2) #used in third autoencoder

2224/2224 [==============================] - 4s 2ms/step


In [24]:
test_au_features2

array([[ 1.73348084e-01, -1.02836885e-01, -8.11585560e-02, ...,
        -2.37917915e-01,  6.51953742e-02,  6.96258470e-02],
       [ 1.42043486e-01,  7.41960853e-02, -8.25248659e-02, ...,
        -2.49009579e-04, -2.62274519e-02, -4.56733033e-02],
       [ 1.71635211e-01, -1.40777647e-01,  4.58459817e-02, ...,
        -9.94885489e-02,  2.50795484e-01, -1.15273468e-01],
       ...,
       [-4.06735837e-02, -1.20183989e-01, -1.12018749e-01, ...,
        -7.35066608e-02,  4.90067936e-02,  2.23591067e-02],
       [ 9.91149992e-02,  6.36094734e-02, -1.83759600e-01, ...,
        -6.72245622e-02,  1.07701711e-01,  2.53769100e-01],
       [ 1.52236521e-02, -5.04268520e-02,  2.19066203e-01, ...,
        -2.15929478e-01,  3.82613428e-02,  6.38305098e-02]], dtype=float32)

encoder 3

In [25]:
input3 = Input(shape=df_autoencode_col3.shape[1:])
encoder = Dense(128)(input3)
encoder = LeakyReLU(alpha=0.15)(encoder)
encoder = Dense(64)(encoder)
encoder = LeakyReLU(alpha=0.15)(encoder)
latent_space3 = Dense(22, activation="tanh")(encoder)

decoder = Dense(64)(latent_space3)
decoder = LeakyReLU(alpha=0.15)(decoder)
decoder = Dense(128)(decoder)
decoder = LeakyReLU(alpha=0.15)(decoder)
decoder = Dense(units=df_autoencode_col3.shape[1], activation="tanh")(decoder)

autoencoder = Model(input3, decoder)
autoencoder.compile(optimizer = "adam",metrics=["mse"],loss="mse")
autoencoder.fit(df_autoencode_col3, df_autoencode_col3, epochs=12, batch_size=25, validation_split=0.2)

Epoch 1/12
2277/2277 [==============================] - 14s 5ms/step - loss: 0.0187 - mse: 0.0187 - val_loss: 0.0118 - val_mse: 0.0118
Epoch 2/12
2277/2277 [==============================] - 10s 4ms/step - loss: 0.0166 - mse: 0.0166 - val_loss: 0.0113 - val_mse: 0.0113
Epoch 3/12
2277/2277 [==============================] - 9s 4ms/step - loss: 0.0163 - mse: 0.0163 - val_loss: 0.0112 - val_mse: 0.0112
Epoch 4/12
2277/2277 [==============================] - 10s 4ms/step - loss: 0.0162 - mse: 0.0162 - val_loss: 0.0112 - val_mse: 0.0112
Epoch 5/12
2277/2277 [==============================] - 10s 5ms/step - loss: 0.0161 - mse: 0.0161 - val_loss: 0.0112 - val_mse: 0.0112
Epoch 6/12
2277/2277 [==============================] - 9s 4ms/step - loss: 0.0161 - mse: 0.0161 - val_loss: 0.0112 - val_mse: 0.0112
Epoch 7/12
2277/2277 [==============================] - 10s 5ms/step - loss: 0.0161 - mse: 0.0161 - val_loss: 0.0112 - val_mse: 0.0112
Epoch 8/12
2277/2277 [==============================] - 1

In [26]:
encoder3 = Model(input3, latent_space3)

In [27]:
test_au_features3 = encoder3.predict(df_autoencode_col3)

2224/2224 [==============================] - 4s 2ms/step


In [28]:
test_au_features3

array([[ 0.18904208, -0.07625757,  0.03791552, ...,  0.04984237,
         0.12495391, -0.1586528 ],
       [-0.00239852, -0.22045627, -0.21921755, ..., -0.00149198,
         0.18918514, -0.1678571 ],
       [ 0.14992674, -0.09402229, -0.19529693, ..., -0.11468276,
         0.12769221, -0.03956058],
       ...,
       [ 0.02880711, -0.07239729, -0.14183894, ..., -0.13691483,
         0.27121735, -0.090159  ],
       [ 0.23765658, -0.05756227,  0.03957856, ..., -0.11664305,
         0.12878232, -0.2551255 ],
       [ 0.2947494 , -0.1719788 , -0.0076583 , ..., -0.02998844,
         0.16827153, -0.03073987]], dtype=float32)

autoencode the autoencoder

In [29]:
dataframe_enc1=pd.DataFrame(test_au_features1, columns=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v'])
dataframe_enc1 #combine featres into a new df

,a,b,c,d,e,f,g,h,i,j,...,m,n,o,p,q,r,s,t,u,v
0,0.325040,0.464541,-0.210951,0.442913,-0.105389,0.052626,-0.357153,-0.085215,-0.022347,-0.294339,...,0.116679,-0.081659,0.190984,-0.254125,0.145661,0.337190,0.234057,0.226714,0.101427,-0.134771
1,0.340931,0.233746,-0.055174,0.433672,-0.059392,0.019311,-0.323846,0.193566,0.167546,-0.149385,...,0.286032,0.193777,0.228870,0.038354,0.167711,-0.067912,0.068059,0.108551,0.062695,0.165206
2,0.245684,0.373550,-0.110773,0.405059,-0.005806,0.080310,-0.279190,-0.145820,0.046768,0.126708,...,0.311150,-0.060466,0.026503,-0.174377,-0.017232,0.224002,0.057027,0.117605,0.169129,-0.034421
3,0.222966,0.281124,0.277333,0.389753,0.239648,-0.261430,-0.215233,0.209327,-0.081553,-0.171139,...,0.264219,0.184182,0.263842,0.142353,-0.347822,-0.395687,-0.150879,-0.099682,-0.096237,0.350401
4,0.283132,0.208308,0.176659,0.481470,0.212387,-0.273782,-0.045497,0.097191,0.042367,0.214147,...,0.466055,0.185819,-0.157045,-0.120312,0.043718,-0.031237,-0.012628,-0.169488,0.087719,0.049735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,0.252721,0.304023,-0.034375,0.437882,0.043398,0.037852,-0.236830,0.027191,-0.067952,0.071627,...,0.275160,-0.069332,0.043303,0.032127,0.045497,0.038084,0.057410,-0.023052,0.071825,0.004314
71147,0.269450,0.256352,-0.010758,0.470749,-0.034567,-0.000988,-0.257180,0.016288,-0.055918,-0.068448,...,0.254024,-0.053964,0.093639,0.014383,0.031805,-0.047897,0.123942,-0.022565,0.014896,-0.018463
71148,0.182470,0.341411,0.204251,0.531795,-0.049875,0.167853,-0.339674,-0.089621,-0.073388,0.129225,...,0.226126,-0.237510,0.225673,-0.203268,0.113102,0.034542,-0.060283,0.240888,0.054095,-0.234455
71149,0.183869,0.070292,0.009340,0.488175,-0.188714,0.037684,-0.247875,0.020141,0.044556,-0.007794,...,0.202371,-0.103989,0.138821,0.168219,-0.238463,-0.035818,0.122703,-0.034948,-0.060764,-0.106884


In [30]:
dataframe_enc2=pd.DataFrame(test_au_features2, columns=['aa','bb','cc','dd','ee','ff','gg','hh','ii','jj','kk','ll','mm','nn','oo','pp','qq','rr','ss','tt','uu','vv'])
dataframe_enc2

,aa,bb,cc,dd,ee,ff,gg,hh,ii,jj,...,mm,nn,oo,pp,qq,rr,ss,tt,uu,vv
0,0.173348,-0.102837,-0.081159,0.106335,-0.183021,0.148846,0.040716,0.356397,-0.169758,-0.231945,...,0.223532,-0.151553,0.082691,-0.158121,0.496854,0.172229,-0.040319,-0.237918,0.065195,0.069626
1,0.142043,0.074196,-0.082525,0.028639,-0.196406,-0.192481,-0.203894,0.130588,0.191522,0.014150,...,0.110828,-0.429956,0.246039,0.227957,0.296924,0.065112,-0.088673,-0.000249,-0.026227,-0.045673
2,0.171635,-0.140778,0.045846,0.049490,0.148765,0.186676,-0.147522,-0.023059,-0.120718,0.031751,...,0.235075,-0.360605,-0.185929,-0.131455,0.431365,0.156146,0.205624,-0.099489,0.250795,-0.115273
3,0.089635,0.102618,0.256846,-0.271475,0.008226,0.078060,-0.046973,-0.011936,0.164487,0.251537,...,0.193635,-0.357673,0.222926,0.231086,0.391837,-0.033711,0.200186,-0.320446,0.145450,0.095167
4,0.026797,0.050985,0.010687,-0.323153,-0.084072,0.151265,-0.180900,0.097331,0.102179,0.017761,...,0.039570,-0.331107,0.059984,0.061985,0.333972,0.029198,0.176182,0.126025,0.113346,0.049180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,0.028190,0.015738,-0.065602,-0.018088,-0.174679,0.016221,-0.063227,0.177738,0.067926,0.004853,...,0.152794,-0.338275,-0.043965,0.052241,0.409723,0.049076,0.016552,-0.094441,0.120083,-0.010954
71147,0.070567,-0.028802,-0.084309,0.055212,-0.121466,-0.041090,-0.016571,0.084131,-0.057408,-0.067506,...,0.182966,-0.341178,-0.090572,0.194493,0.438673,0.094389,-0.031237,0.005655,0.225713,-0.093971
71148,-0.040674,-0.120184,-0.112019,0.109231,-0.110276,0.176360,-0.145409,0.209000,-0.052740,-0.045260,...,0.101730,-0.410912,-0.430677,0.187219,0.423155,-0.120868,-0.022192,-0.073507,0.049007,0.022359
71149,0.099115,0.063609,-0.183760,-0.162016,-0.274326,0.130859,-0.149429,-0.013962,-0.174076,-0.073605,...,0.116603,-0.336377,-0.265093,0.251475,0.352029,-0.134710,0.121782,-0.067225,0.107702,0.253769


In [31]:
dataframe_enc3=pd.DataFrame(test_au_features3, columns=['a3','b3','c3','d3','e3','f3','g3','h3','i3','j3','k3','l3','m3','n3','o3','p3','q3','r3','s3','t3','u3','v3'])
dataframe_enc3

,a3,b3,c3,d3,e3,f3,g3,h3,i3,j3,...,m3,n3,o3,p3,q3,r3,s3,t3,u3,v3
0,0.189042,-0.076258,0.037916,0.382305,-0.161034,-0.071770,-0.101459,0.132276,-0.366636,-0.172509,...,-0.489878,0.037257,0.024535,0.338292,0.013204,0.075113,-0.003557,0.049842,0.124954,-0.158653
1,-0.002399,-0.220456,-0.219218,0.339893,-0.284552,-0.020100,0.299357,0.028528,-0.336097,-0.164115,...,-0.359794,-0.214291,-0.017730,0.157638,0.227396,-0.111950,0.087194,-0.001492,0.189185,-0.167857
2,0.149927,-0.094022,-0.195297,0.306823,0.057172,0.130738,0.073188,0.072289,-0.324619,-0.055698,...,-0.476921,-0.036959,-0.141694,0.043374,0.010759,0.115578,0.035217,-0.114683,0.127692,-0.039561
3,-0.110018,-0.155190,-0.109278,0.134790,-0.107253,0.006484,-0.301684,-0.199815,-0.175955,-0.008965,...,-0.469867,0.467831,0.073431,0.207289,-0.030316,0.276375,0.059614,-0.234658,-0.159824,-0.181343
4,0.112995,-0.169937,0.133022,0.375335,-0.092117,-0.021338,0.086047,0.113271,-0.208208,0.160198,...,-0.392773,0.107117,0.169788,0.195965,-0.119011,-0.014887,-0.009000,-0.135337,0.225235,-0.160912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,0.042814,-0.091567,-0.011359,0.389601,-0.084869,-0.050766,-0.014713,-0.050897,-0.220553,0.006678,...,-0.376665,0.022204,-0.006909,-0.013519,-0.065290,0.049160,0.088865,-0.168577,0.104935,-0.114757
71147,0.074133,-0.093445,-0.213637,0.323156,0.048728,0.012289,-0.060792,-0.027687,-0.158987,0.026967,...,-0.417484,-0.035909,-0.012973,-0.024999,0.018144,0.056751,0.076039,-0.197652,0.126869,-0.044049
71148,0.028807,-0.072397,-0.141839,0.536369,-0.132407,-0.234598,0.014289,0.202138,-0.163547,-0.233759,...,-0.384099,-0.103512,-0.085758,-0.082730,0.039550,0.073782,0.030276,-0.136915,0.271217,-0.090159
71149,0.237657,-0.057562,0.039579,0.388872,-0.113076,-0.004017,-0.079168,0.054345,-0.074037,0.142480,...,-0.449479,0.222943,0.111501,-0.010681,-0.094772,0.017242,-0.118537,-0.116643,0.128782,-0.255125


In [32]:
df_encoder_combined = pd.concat([dataframe_enc1, dataframe_enc2,dataframe_enc3], axis=1)
df_encoder_combined #take all the encoded values into a new df

,a,b,c,d,e,f,g,h,i,j,...,m3,n3,o3,p3,q3,r3,s3,t3,u3,v3
0,0.325040,0.464541,-0.210951,0.442913,-0.105389,0.052626,-0.357153,-0.085215,-0.022347,-0.294339,...,-0.489878,0.037257,0.024535,0.338292,0.013204,0.075113,-0.003557,0.049842,0.124954,-0.158653
1,0.340931,0.233746,-0.055174,0.433672,-0.059392,0.019311,-0.323846,0.193566,0.167546,-0.149385,...,-0.359794,-0.214291,-0.017730,0.157638,0.227396,-0.111950,0.087194,-0.001492,0.189185,-0.167857
2,0.245684,0.373550,-0.110773,0.405059,-0.005806,0.080310,-0.279190,-0.145820,0.046768,0.126708,...,-0.476921,-0.036959,-0.141694,0.043374,0.010759,0.115578,0.035217,-0.114683,0.127692,-0.039561
3,0.222966,0.281124,0.277333,0.389753,0.239648,-0.261430,-0.215233,0.209327,-0.081553,-0.171139,...,-0.469867,0.467831,0.073431,0.207289,-0.030316,0.276375,0.059614,-0.234658,-0.159824,-0.181343
4,0.283132,0.208308,0.176659,0.481470,0.212387,-0.273782,-0.045497,0.097191,0.042367,0.214147,...,-0.392773,0.107117,0.169788,0.195965,-0.119011,-0.014887,-0.009000,-0.135337,0.225235,-0.160912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,0.252721,0.304023,-0.034375,0.437882,0.043398,0.037852,-0.236830,0.027191,-0.067952,0.071627,...,-0.376665,0.022204,-0.006909,-0.013519,-0.065290,0.049160,0.088865,-0.168577,0.104935,-0.114757
71147,0.269450,0.256352,-0.010758,0.470749,-0.034567,-0.000988,-0.257180,0.016288,-0.055918,-0.068448,...,-0.417484,-0.035909,-0.012973,-0.024999,0.018144,0.056751,0.076039,-0.197652,0.126869,-0.044049
71148,0.182470,0.341411,0.204251,0.531795,-0.049875,0.167853,-0.339674,-0.089621,-0.073388,0.129225,...,-0.384099,-0.103512,-0.085758,-0.082730,0.039550,0.073782,0.030276,-0.136915,0.271217,-0.090159
71149,0.183869,0.070292,0.009340,0.488175,-0.188714,0.037684,-0.247875,0.020141,0.044556,-0.007794,...,-0.449479,0.222943,0.111501,-0.010681,-0.094772,0.017242,-0.118537,-0.116643,0.128782,-0.255125


In [33]:
inputFinal = Input(shape=df_encoder_combined.shape[1:])
encoder = Dense(48)(inputFinal)
encoder = LeakyReLU(alpha=0.15)(encoder)
encoder = Dense(32)(encoder)
encoder = LeakyReLU(alpha=0.15)(encoder)
latent_spaceFinal = Dense(16, activation="tanh")(encoder)

decoder = Dense(32)(latent_spaceFinal)
decoder = LeakyReLU(alpha=0.15)(decoder)
decoder = Dense(48)(decoder)
decoder = LeakyReLU(alpha=0.15)(decoder)
decoder = Dense(units=df_encoder_combined.shape[1], activation="tanh")(decoder)

autoencoder = Model(inputFinal, decoder)
autoencoder.compile(optimizer = "adam",metrics=["mse"],loss="mse")
autoencoder.fit(df_encoder_combined, df_encoder_combined, epochs=12, batch_size=25, validation_split=0.22)

Epoch 1/12
2220/2220 [==============================] - 12s 5ms/step - loss: 0.0073 - mse: 0.0073 - val_loss: 0.0043 - val_mse: 0.0043
Epoch 2/12
2220/2220 [==============================] - 10s 5ms/step - loss: 0.0060 - mse: 0.0060 - val_loss: 0.0041 - val_mse: 0.0041
Epoch 3/12
2220/2220 [==============================] - 9s 4ms/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.0041 - val_mse: 0.0041
Epoch 4/12
2220/2220 [==============================] - 10s 5ms/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.0041 - val_mse: 0.0041
Epoch 5/12
2220/2220 [==============================] - 12s 5ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.0040 - val_mse: 0.0040
Epoch 6/12
2220/2220 [==============================] - 9s 4ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.0040 - val_mse: 0.0040
Epoch 7/12
2220/2220 [==============================] - 10s 4ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.0040 - val_mse: 0.0040
Epoch 8/12
2220/2220 [==============================] - 1

take the latent space

In [34]:
encoderFinal = Model(inputFinal, latent_spaceFinal)

In [35]:
test_au_featuresFinal = encoderFinal.predict(df_encoder_combined)


2224/2224 [==============================] - 3s 1ms/step


In [36]:
test_au_featuresFinal

array([[ 0.20837858,  0.13925405,  0.02208015, ...,  0.20397495,
        -0.00249175, -0.1616738 ],
       [ 0.19773704,  0.02297232,  0.00875038, ..., -0.01893319,
         0.22138846, -0.2161164 ],
       [ 0.08577849,  0.15072425, -0.11846703, ...,  0.14792474,
        -0.03791278, -0.05587339],
       ...,
       [ 0.0549386 ,  0.14487967, -0.0488093 , ...,  0.19054753,
         0.02675739, -0.20206016],
       [ 0.10191461,  0.0724726 , -0.03159774, ...,  0.05931821,
        -0.09820942, -0.080561  ],
       [ 0.01091855,  0.03969748, -0.07647774, ..., -0.01418877,
        -0.13102879, -0.02749566]], dtype=float32)

In [37]:
dataframe_encoder_final=pd.DataFrame(test_au_featuresFinal, columns=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p'])
dataframe_encoder_final

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p
0,0.208379,0.139254,0.022080,0.080784,-0.169311,-0.069097,-0.037997,-0.126047,-0.156844,0.082969,-0.010569,0.149803,-0.055943,0.203975,-0.002492,-0.161674
1,0.197737,0.022972,0.008750,0.239600,0.003906,-0.149550,0.016080,-0.097651,-0.004595,0.224654,0.113765,0.013481,0.076214,-0.018933,0.221388,-0.216116
2,0.085778,0.150724,-0.118467,0.253787,-0.076099,-0.044557,-0.033397,0.077216,-0.023355,-0.049063,0.086096,-0.062975,0.112927,0.147925,-0.037913,-0.055873
3,0.339914,-0.161968,0.237728,0.145624,0.098367,-0.068587,0.100288,0.020596,0.076030,-0.188622,0.159737,0.186299,0.054227,-0.214219,-0.092772,0.023308
4,0.241230,0.037504,0.027847,0.095753,0.023207,-0.063613,0.034309,0.086764,0.110336,-0.024749,0.094043,-0.022807,0.065303,-0.122614,0.106364,0.078460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,0.134037,0.105709,0.037506,0.181399,-0.018655,-0.027156,0.037436,-0.002368,0.017656,0.010512,0.074435,0.004199,0.074189,-0.015373,0.009484,-0.066338
71147,0.099121,0.077715,-0.053535,0.182138,0.034615,0.007397,0.027726,-0.048090,-0.021493,0.064414,0.104140,-0.027435,0.067389,0.010273,0.008514,-0.080273
71148,0.054939,0.144880,-0.048809,0.180603,0.028120,0.040966,-0.067093,0.072180,-0.120755,-0.041118,0.075452,-0.166895,0.139777,0.190548,0.026757,-0.202060
71149,0.101915,0.072473,-0.031598,0.150281,0.068915,0.179135,0.018302,-0.047174,0.103606,-0.009846,0.044892,0.075918,-0.029220,0.059318,-0.098209,-0.080561


In [38]:
dataframe_encoder_final.iloc[[15240]]

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p
15240,-0.009802,0.078289,-0.027817,0.137227,0.053756,-0.169485,0.015644,-0.051171,0.028289,0.023874,0.078537,-0.035362,0.16884,-0.029276,0.008665,-0.052691


In [39]:
df_Only_non_sequential = df_data_better[['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre','Duration','Danceability', 'Energy','Key','Loudness','Speechiness','Acousticness','Instrumentalness','Liveness','Valence','Tempo','TimeSignature','ZeroCrossing','MFCCSMax','MFCCSMin','MFCCSMean','MFCCSMedian','Rms','Bpm','RolloffMin','RolloffMax','ChromaMedian','ChromaMean','ChromaMin','ChromaMax']]
df_Only_non_sequential #jugglint the dataframes so that they have the correct data for training

,TrackID,TrackName,Album,PreviewURL,ArtistName,Genre,Duration,Danceability,Energy,Key,...,MFCCSMean,MFCCSMedian,Rms,Bpm,RolloffMin,RolloffMax,ChromaMedian,ChromaMean,ChromaMin,ChromaMax
0,37Jdi2cCSRA5jl6zADeLOy,D R O V E MY CAR,1 Trait High,https://p.scdn.co/mp3-preview/ab54f8f789175d42...,1 Trait Danger,comedy rap,120937,0.601,0.827,2,...,5.030483,3.302809,0.304123,128.129013,73.953094,6760.593052,0.349420,0.421962,0.005061,1
1,14jLrcO0NGIxtX56LHEhsv,Reprisal,1 Trait World Tour,https://p.scdn.co/mp3-preview/ffe73947f36e6e15...,1 Trait Danger,comedy rap,100634,0.391,0.575,5,...,3.887536,3.168254,0.285675,72.090485,197.802658,7413.242569,0.478386,0.531803,0.053862,1
2,7dcNDXD9SwJ4Txqg2fE2t8,Danger,Back from the Dead,https://p.scdn.co/mp3-preview/1ae38bb85e0313f6...,LUCKY3RD,houston rap,181395,0.802,0.511,6,...,1.650047,2.126344,0.297220,84.936028,185.067787,6498.745937,0.329896,0.406566,0.001817,1
3,0ackjn8zBcLQnkeRPeOCVj,Danger,Danger,https://p.scdn.co/mp3-preview/ab856e3e92deddc1...,Cooliecut,dark trap,96052,0.720,0.569,5,...,5.127454,3.298813,0.405741,120.059700,127.567406,5817.598572,0.521628,0.536370,0.011882,1
4,1wDY9VWdCoRjLV4gUyYf52,Danger,In Transit (Deluxe),https://p.scdn.co/mp3-preview/61586296e9713d07...,Terri,nigerian pop,157648,0.514,0.582,11,...,1.093476,-0.297896,0.299387,97.207802,141.000423,5296.091309,0.248856,0.346003,0.001743,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,75dwXgnlK10c8YtvUODVbj,My Foolish Heart,Everything I Have Is Yours / The Best Of The M...,https://p.scdn.co/mp3-preview/15f2927aa3ef44ae...,Billy Eckstine,adult standards,185906,0.316,0.110,9,...,-5.347975,-3.515098,0.096891,96.391342,218.175087,2585.977879,0.069974,0.215599,0.000245,1
71147,4f9Zf8gkeZ5NuSRyElXa7r,Alalaga,Samoa With Love,https://p.scdn.co/mp3-preview/df69922b04c63104...,The Five Stars,pacific islands pop,518666,0.355,0.362,1,...,-2.967051,-3.451842,0.147721,85.702225,155.476456,3527.954578,0.191280,0.296154,0.001543,1
71148,3e4OUcQE9tTujfgT0MbKq0,Feel The Fire - Futuristic Polar Bears & Jerry...,Feel The Fire (The Remixes),https://p.scdn.co/mp3-preview/9c6fb71eaac48cca...,MR.BLACK,big room,161904,0.591,0.973,1,...,3.457322,0.285472,0.333917,126.038162,242.130775,7507.046585,0.433053,0.488847,0.041757,1
71149,2nnMT7bf8F5xBYOBzZiDZC,Blast Off To Nowhere,Tonight The Stars Revolt,https://p.scdn.co/mp3-preview/221e0fa36c554556...,Powerman 5000,alternative metal,225293,0.522,0.988,1,...,2.860447,-0.272149,0.282717,142.889572,220.967674,6753.830280,0.480341,0.522678,0.039996,1


In [40]:
df_non_seq_and_autoencoded = pd.concat([df_Only_non_sequential, dataframe_encoder_final], axis=1) #this is df_better_data with encoded values 
df_non_seq_and_autoencoded

,TrackID,TrackName,Album,PreviewURL,ArtistName,Genre,Duration,Danceability,Energy,Key,...,g,h,i,j,k,l,m,n,o,p
0,37Jdi2cCSRA5jl6zADeLOy,D R O V E MY CAR,1 Trait High,https://p.scdn.co/mp3-preview/ab54f8f789175d42...,1 Trait Danger,comedy rap,120937,0.601,0.827,2,...,-0.037997,-0.126047,-0.156844,0.082969,-0.010569,0.149803,-0.055943,0.203975,-0.002492,-0.161674
1,14jLrcO0NGIxtX56LHEhsv,Reprisal,1 Trait World Tour,https://p.scdn.co/mp3-preview/ffe73947f36e6e15...,1 Trait Danger,comedy rap,100634,0.391,0.575,5,...,0.016080,-0.097651,-0.004595,0.224654,0.113765,0.013481,0.076214,-0.018933,0.221388,-0.216116
2,7dcNDXD9SwJ4Txqg2fE2t8,Danger,Back from the Dead,https://p.scdn.co/mp3-preview/1ae38bb85e0313f6...,LUCKY3RD,houston rap,181395,0.802,0.511,6,...,-0.033397,0.077216,-0.023355,-0.049063,0.086096,-0.062975,0.112927,0.147925,-0.037913,-0.055873
3,0ackjn8zBcLQnkeRPeOCVj,Danger,Danger,https://p.scdn.co/mp3-preview/ab856e3e92deddc1...,Cooliecut,dark trap,96052,0.720,0.569,5,...,0.100288,0.020596,0.076030,-0.188622,0.159737,0.186299,0.054227,-0.214219,-0.092772,0.023308
4,1wDY9VWdCoRjLV4gUyYf52,Danger,In Transit (Deluxe),https://p.scdn.co/mp3-preview/61586296e9713d07...,Terri,nigerian pop,157648,0.514,0.582,11,...,0.034309,0.086764,0.110336,-0.024749,0.094043,-0.022807,0.065303,-0.122614,0.106364,0.078460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,75dwXgnlK10c8YtvUODVbj,My Foolish Heart,Everything I Have Is Yours / The Best Of The M...,https://p.scdn.co/mp3-preview/15f2927aa3ef44ae...,Billy Eckstine,adult standards,185906,0.316,0.110,9,...,0.037436,-0.002368,0.017656,0.010512,0.074435,0.004199,0.074189,-0.015373,0.009484,-0.066338
71147,4f9Zf8gkeZ5NuSRyElXa7r,Alalaga,Samoa With Love,https://p.scdn.co/mp3-preview/df69922b04c63104...,The Five Stars,pacific islands pop,518666,0.355,0.362,1,...,0.027726,-0.048090,-0.021493,0.064414,0.104140,-0.027435,0.067389,0.010273,0.008514,-0.080273
71148,3e4OUcQE9tTujfgT0MbKq0,Feel The Fire - Futuristic Polar Bears & Jerry...,Feel The Fire (The Remixes),https://p.scdn.co/mp3-preview/9c6fb71eaac48cca...,MR.BLACK,big room,161904,0.591,0.973,1,...,-0.067093,0.072180,-0.120755,-0.041118,0.075452,-0.166895,0.139777,0.190548,0.026757,-0.202060
71149,2nnMT7bf8F5xBYOBzZiDZC,Blast Off To Nowhere,Tonight The Stars Revolt,https://p.scdn.co/mp3-preview/221e0fa36c554556...,Powerman 5000,alternative metal,225293,0.522,0.988,1,...,0.018302,-0.047174,0.103606,-0.009846,0.044892,0.075918,-0.029220,0.059318,-0.098209,-0.080561


In [41]:
df_first_cluster = df_non_seq_and_autoencoded.copy()

In [42]:
df_first_cluster = df_first_cluster.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre','Duration'], axis=1) #drop these for training on som
df_first_cluster

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,...,g,h,i,j,k,l,m,n,o,p
0,0.601,0.827,2,-7.056,0.1350,0.01180,0.000000,0.9610,0.2930,128.055,...,-0.037997,-0.126047,-0.156844,0.082969,-0.010569,0.149803,-0.055943,0.203975,-0.002492,-0.161674
1,0.391,0.575,5,-7.520,0.1070,0.38000,0.004660,0.4130,0.3260,143.260,...,0.016080,-0.097651,-0.004595,0.224654,0.113765,0.013481,0.076214,-0.018933,0.221388,-0.216116
2,0.802,0.511,6,-9.651,0.2980,0.03120,0.000000,0.0843,0.3320,84.979,...,-0.033397,0.077216,-0.023355,-0.049063,0.086096,-0.062975,0.112927,0.147925,-0.037913,-0.055873
3,0.720,0.569,5,-6.983,0.0607,0.01650,0.000525,0.5400,0.1690,119.989,...,0.100288,0.020596,0.076030,-0.188622,0.159737,0.186299,0.054227,-0.214219,-0.092772,0.023308
4,0.514,0.582,11,-5.243,0.3880,0.35500,0.000001,0.1840,0.4530,66.681,...,0.034309,0.086764,0.110336,-0.024749,0.094043,-0.022807,0.065303,-0.122614,0.106364,0.078460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,0.316,0.110,9,-17.251,0.0367,0.99200,0.539000,0.4030,0.1220,88.791,...,0.037436,-0.002368,0.017656,0.010512,0.074435,0.004199,0.074189,-0.015373,0.009484,-0.066338
71147,0.355,0.362,1,-13.699,0.0340,0.70900,0.623000,0.1220,0.3570,86.946,...,0.027726,-0.048090,-0.021493,0.064414,0.104140,-0.027435,0.067389,0.010273,0.008514,-0.080273
71148,0.591,0.973,1,-2.835,0.0471,0.00420,0.390000,0.1850,0.0419,125.974,...,-0.067093,0.072180,-0.120755,-0.041118,0.075452,-0.166895,0.139777,0.190548,0.026757,-0.202060
71149,0.522,0.988,1,-4.646,0.3210,0.00320,0.051300,0.1200,0.0689,141.685,...,0.018302,-0.047174,0.103606,-0.009846,0.044892,0.075918,-0.029220,0.059318,-0.098209,-0.080561


In [43]:
df_first_cluster=sc.fit_transform(df_first_cluster)
df_first_cluster

array([[0.60953347, 0.827     , 0.18181818, ..., 0.69614572, 0.48363502,
        0.36474546],
       [0.39655172, 0.575     , 0.45454545, ..., 0.43450308, 0.73171363,
        0.28692147],
       [0.81338742, 0.511     , 0.54545455, ..., 0.63035572, 0.44438546,
        0.51598385],
       ...,
       [0.59939148, 0.973     , 0.09090909, ..., 0.68038503, 0.5160456 ,
        0.30701442],
       [0.52941176, 0.988     , 0.09090909, ..., 0.52635214, 0.37757158,
        0.48069369],
       [0.94827586, 0.484     , 0.45454545, ..., 0.44007193, 0.34120489,
        0.55654893]])

In [44]:
songsom = SOM(m=11, n=4, dim=40) #create SOM with 44 nodes
songsom.fit(df_first_cluster) #fit som on values

In [45]:
predictions = songsom.predict(df_first_cluster) #cluster predictions
predictions


array([15, 19, 23, ...,  6,  9,  3])

In [46]:
max(predictions)

43

In [47]:
len(np.unique(predictions))


44

In [48]:
#df_data_better = pd.DataFrame(test_au_features)
df_non_seq_and_autoencoded.insert(47, "clusterLabels", predictions, True) #insert at 47 in actual run

In [49]:
df_non_seq_and_autoencoded

,TrackID,TrackName,Album,PreviewURL,ArtistName,Genre,Duration,Danceability,Energy,Key,...,h,i,j,k,l,m,n,o,p,clusterLabels
0,37Jdi2cCSRA5jl6zADeLOy,D R O V E MY CAR,1 Trait High,https://p.scdn.co/mp3-preview/ab54f8f789175d42...,1 Trait Danger,comedy rap,120937,0.601,0.827,2,...,-0.126047,-0.156844,0.082969,-0.010569,0.149803,-0.055943,0.203975,-0.002492,-0.161674,15
1,14jLrcO0NGIxtX56LHEhsv,Reprisal,1 Trait World Tour,https://p.scdn.co/mp3-preview/ffe73947f36e6e15...,1 Trait Danger,comedy rap,100634,0.391,0.575,5,...,-0.097651,-0.004595,0.224654,0.113765,0.013481,0.076214,-0.018933,0.221388,-0.216116,19
2,7dcNDXD9SwJ4Txqg2fE2t8,Danger,Back from the Dead,https://p.scdn.co/mp3-preview/1ae38bb85e0313f6...,LUCKY3RD,houston rap,181395,0.802,0.511,6,...,0.077216,-0.023355,-0.049063,0.086096,-0.062975,0.112927,0.147925,-0.037913,-0.055873,23
3,0ackjn8zBcLQnkeRPeOCVj,Danger,Danger,https://p.scdn.co/mp3-preview/ab856e3e92deddc1...,Cooliecut,dark trap,96052,0.720,0.569,5,...,0.020596,0.076030,-0.188622,0.159737,0.186299,0.054227,-0.214219,-0.092772,0.023308,0
4,1wDY9VWdCoRjLV4gUyYf52,Danger,In Transit (Deluxe),https://p.scdn.co/mp3-preview/61586296e9713d07...,Terri,nigerian pop,157648,0.514,0.582,11,...,0.086764,0.110336,-0.024749,0.094043,-0.022807,0.065303,-0.122614,0.106364,0.078460,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,75dwXgnlK10c8YtvUODVbj,My Foolish Heart,Everything I Have Is Yours / The Best Of The M...,https://p.scdn.co/mp3-preview/15f2927aa3ef44ae...,Billy Eckstine,adult standards,185906,0.316,0.110,9,...,-0.002368,0.017656,0.010512,0.074435,0.004199,0.074189,-0.015373,0.009484,-0.066338,41
71147,4f9Zf8gkeZ5NuSRyElXa7r,Alalaga,Samoa With Love,https://p.scdn.co/mp3-preview/df69922b04c63104...,The Five Stars,pacific islands pop,518666,0.355,0.362,1,...,-0.048090,-0.021493,0.064414,0.104140,-0.027435,0.067389,0.010273,0.008514,-0.080273,37
71148,3e4OUcQE9tTujfgT0MbKq0,Feel The Fire - Futuristic Polar Bears & Jerry...,Feel The Fire (The Remixes),https://p.scdn.co/mp3-preview/9c6fb71eaac48cca...,MR.BLACK,big room,161904,0.591,0.973,1,...,0.072180,-0.120755,-0.041118,0.075452,-0.166895,0.139777,0.190548,0.026757,-0.202060,6
71149,2nnMT7bf8F5xBYOBzZiDZC,Blast Off To Nowhere,Tonight The Stars Revolt,https://p.scdn.co/mp3-preview/221e0fa36c554556...,Powerman 5000,alternative metal,225293,0.522,0.988,1,...,-0.047174,0.103606,-0.009846,0.044892,0.075918,-0.029220,0.059318,-0.098209,-0.080561,9


In [50]:
df_clust0 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 0] #this next section turns each cluster into a dataframe
len(df_clust0)

1672

In [51]:
df_clust1 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 1]
len(df_clust1)

1126

In [52]:
df_clust2 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 2]
len(df_clust2)

1548

In [53]:
df_clust3 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 3]
len(df_clust3)

1804

In [54]:
df_clust4 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 4]
len(df_clust4)

1664

In [55]:
df_clust5 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 5]
len(df_clust5)

743

In [56]:
df_clust6 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 6]
len(df_clust6)

951

In [57]:
df_clust7 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 7]
len(df_clust7)

1790

In [58]:
df_clust8 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 8]
len(df_clust8)

1664

In [59]:
df_clust9 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 9]
len(df_clust9)

1612

In [60]:
df_clust10 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 10]
len(df_clust10)

810

In [61]:
df_clust11 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 11]
len(df_clust11)

494

In [62]:
df_clust12 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 12]
len(df_clust12)

2777

In [63]:
df_clust13 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 13]
len(df_clust13)

1122

In [64]:
df_clust14 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 14]
len(df_clust14)

890

In [65]:
df_clust15 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 15]
len(df_clust15)

2429

In [66]:
df_clust16 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 16]
len(df_clust16)

1508

In [67]:
df_clust17 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 17]
len(df_clust17)

1927

In [68]:
df_clust18 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 18]
len(df_clust18)

2563

In [69]:
df_clust19 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 19]


In [70]:
df_clust20 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 20]


In [71]:
df_clust21 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 21]


In [72]:
df_clust22 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 22]


In [73]:
df_clust23 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 23]


In [74]:
df_clust24 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 24]


In [75]:
df_clust25 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 25]


In [76]:
df_clust26 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 26]


In [77]:
df_clust27 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 27]


In [78]:
df_clust28 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 28]


In [79]:
df_clust29 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 29]


In [80]:
df_clust30 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 30]


In [81]:
df_clust31 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 31]


In [82]:
df_clust32 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 32]


In [83]:
df_clust33 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 33]


In [84]:
df_clust34 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 34]


In [85]:
df_clust35 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 35]


In [86]:
df_clust36 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 36]


In [87]:
df_clust37 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 37]


In [88]:
df_clust38 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 38]
len(df_clust38)

521

In [89]:
df_clust39 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 39]
len(df_clust39)

1182

In [90]:
df_clust40 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 40]
len(df_clust40)

1111

In [91]:
df_clust42 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 42]
len(df_clust42)

1367

In [92]:
df_clust41 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 41]
len(df_clust41)

2207

In [93]:
df_clust43 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 43]
len(df_clust43)

2007

In [94]:
df_clust44 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 44] 
len(df_clust44)

0

Make copies and redrop values

In [95]:
df_clust44_big = df_clust44.copy() #make copies to redrop values
df_clust0_big = df_clust0.copy()
df_clust1_big = df_clust1.copy()
df_clust2_big = df_clust2.copy()
df_clust3_big = df_clust3.copy()
df_clust4_big = df_clust4.copy()
df_clust5_big = df_clust5.copy()
df_clust6_big = df_clust6.copy()
df_clust7_big = df_clust7.copy()
df_clust8_big = df_clust8.copy()
df_clust9_big = df_clust9.copy()
df_clust10_big = df_clust10.copy()
df_clust11_big = df_clust11.copy()
df_clust12_big = df_clust12.copy()
df_clust13_big = df_clust13.copy()
df_clust14_big = df_clust14.copy()
df_clust15_big = df_clust15.copy()
df_clust16_big = df_clust16.copy()
df_clust17_big = df_clust17.copy()
df_clust18_big = df_clust18.copy()
df_clust19_big = df_clust19.copy()
df_clust20_big = df_clust20.copy()
df_clust21_big = df_clust21.copy()
df_clust22_big = df_clust22.copy()
df_clust23_big = df_clust23.copy()
df_clust24_big = df_clust24.copy()
df_clust25_big = df_clust25.copy()
df_clust26_big = df_clust26.copy()
df_clust27_big = df_clust27.copy()
df_clust28_big = df_clust28.copy()
df_clust29_big = df_clust29.copy()
df_clust30_big = df_clust30.copy()
df_clust31_big = df_clust31.copy()
df_clust32_big = df_clust32.copy()
df_clust33_big = df_clust33.copy()
df_clust34_big = df_clust34.copy()
df_clust35_big = df_clust35.copy()
df_clust36_big = df_clust36.copy()
df_clust37_big = df_clust37.copy()
df_clust38_big = df_clust38.copy()
df_clust39_big = df_clust39.copy()
df_clust40_big = df_clust40.copy()
df_clust41_big = df_clust41.copy()
df_clust42_big = df_clust42.copy()
df_clust43_big = df_clust43.copy()

In [96]:
df_clust0 = df_clust0.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1) #drop values which cant be trained 
df_clust1 = df_clust1.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust2 = df_clust2.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust3 = df_clust3.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust4 = df_clust4.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust5 = df_clust5.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust6 = df_clust6.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust7 = df_clust7.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust8 = df_clust8.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust9 = df_clust9.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust10 = df_clust10.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust11 = df_clust11.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust12 = df_clust12.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust13 = df_clust13.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust14 = df_clust14.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust15 = df_clust15.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust16 = df_clust16.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust17 = df_clust17.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust18 = df_clust18.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust19 = df_clust19.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust20 = df_clust20.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust21 = df_clust21.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust22 = df_clust22.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust23 = df_clust23.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust24 = df_clust24.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust25 = df_clust25.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust26 = df_clust26.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust27 = df_clust27.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust28 = df_clust28.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust29 = df_clust29.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust30 = df_clust30.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust31 = df_clust31.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust32 = df_clust32.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust33 = df_clust33.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust34 = df_clust34.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust35 = df_clust35.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust36 = df_clust36.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust37 = df_clust37.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust38 = df_clust38.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust39 = df_clust39.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust40 = df_clust40.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust41 = df_clust41.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust42 = df_clust42.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust43 = df_clust43.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)

In [97]:
df_clust44 = df_clust44.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)


In [98]:
df_clust44

,Duration,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,...,h,i,j,k,l,m,n,o,p,clusterLabels


distance calculation stuff

## Clustering the clusters


In [99]:

#df_clust44=sc.fit_transform(df_clust44)



In [100]:
df_clust0=sc.fit_transform(df_clust0)#do cluster calculations on each of the bigger clusters to make subclusters
df_clust0
df_clust1=sc.fit_transform(df_clust1)
df_clust1
df_clust2=sc.fit_transform(df_clust2)
df_clust2
df_clust3=sc.fit_transform(df_clust3)
df_clust3
df_clust4=sc.fit_transform(df_clust4)
df_clust4
df_clust5=sc.fit_transform(df_clust5)
df_clust5
df_clust6=sc.fit_transform(df_clust6)
df_clust6
df_clust7=sc.fit_transform(df_clust7)
df_clust7

array([[0.09282868, 0.32405345, 0.579974  , ..., 0.38652071, 0.49964012,
        0.        ],
       [0.18170648, 0.4610245 , 0.98199889, ..., 0.41423598, 0.41207351,
        0.        ],
       [0.16056251, 0.3986637 , 0.97599851, ..., 0.59669527, 0.44035795,
        0.        ],
       ...,
       [0.10442531, 0.22828508, 0.58097406, ..., 0.59434609, 0.66628264,
        0.        ],
       [0.12290928, 0.54899777, 0.52697072, ..., 0.56083442, 0.59956139,
        0.        ],
       [0.05977285, 0.42650334, 0.89199331, ..., 0.55420631, 0.36610907,
        0.        ]])

In [101]:
df_clust8=sc.fit_transform(df_clust8)
df_clust8
df_clust9=sc.fit_transform(df_clust9)
df_clust9
df_clust10=sc.fit_transform(df_clust10)
df_clust10
df_clust11=sc.fit_transform(df_clust11)
df_clust11
df_clust12=sc.fit_transform(df_clust12)
df_clust12
df_clust13=sc.fit_transform(df_clust13)
df_clust13
df_clust14=sc.fit_transform(df_clust14)
df_clust14
df_clust15=sc.fit_transform(df_clust15)
df_clust15
df_clust16=sc.fit_transform(df_clust16)
df_clust16
df_clust17=sc.fit_transform(df_clust17)
df_clust17
df_clust18=sc.fit_transform(df_clust18)
df_clust18
df_clust19=sc.fit_transform(df_clust19)
df_clust19
df_clust20=sc.fit_transform(df_clust20)
df_clust20
df_clust21=sc.fit_transform(df_clust21)
df_clust21
df_clust22=sc.fit_transform(df_clust22)
df_clust22
df_clust23=sc.fit_transform(df_clust23)
df_clust23
df_clust24=sc.fit_transform(df_clust24)
df_clust24
df_clust25=sc.fit_transform(df_clust25)
df_clust25
df_clust26=sc.fit_transform(df_clust26)
df_clust26
df_clust27=sc.fit_transform(df_clust27)
df_clust27
df_clust28=sc.fit_transform(df_clust28)
df_clust28
df_clust29=sc.fit_transform(df_clust29)
df_clust29
df_clust30=sc.fit_transform(df_clust30)
df_clust30
df_clust31=sc.fit_transform(df_clust31)
df_clust31
df_clust32=sc.fit_transform(df_clust32)
df_clust32
df_clust33=sc.fit_transform(df_clust33)
df_clust33
df_clust34=sc.fit_transform(df_clust34)
df_clust34
df_clust35=sc.fit_transform(df_clust35)
df_clust35
df_clust36=sc.fit_transform(df_clust36)
df_clust36
df_clust37=sc.fit_transform(df_clust37)
df_clust37


array([[0.02566789, 0.21919585, 0.50776583, ..., 0.80697515, 0.43085356,
        0.        ],
       [0.04591955, 0.80155642, 0.31063321, ..., 0.74652985, 0.40551417,
        0.        ],
       [0.16493474, 0.39559014, 0.1995221 , ..., 0.52810471, 0.49012956,
        0.        ],
       ...,
       [0.12243321, 0.49546044, 0.52807646, ..., 0.52590055, 0.3206999 ,
        0.        ],
       [0.00695499, 0.60181582, 0.48864994, ..., 0.4886572 , 0.38414344,
        0.        ],
       [0.52067618, 0.25291829, 0.30585424, ..., 0.48736528, 0.39374323,
        0.        ]])

In [102]:
df_clust38=sc.fit_transform(df_clust38)
df_clust38
df_clust39=sc.fit_transform(df_clust39)
df_clust39
df_clust40=sc.fit_transform(df_clust40)
df_clust40
df_clust41=sc.fit_transform(df_clust41)
df_clust41
df_clust42=sc.fit_transform(df_clust42)
df_clust42
df_clust43=sc.fit_transform(df_clust43)
df_clust43

array([[0.1764207 , 0.46217617, 0.02786682, ..., 0.42767747, 0.56658781,
        0.        ],
       [0.00831902, 0.60414508, 0.004374  , ..., 0.45966437, 0.55799005,
        0.        ],
       [0.07466304, 0.53056995, 0.33013874, ..., 0.56136845, 0.5697825 ,
        0.        ],
       ...,
       [0.02331675, 0.21554404, 0.01392131, ..., 0.51762399, 0.53317302,
        0.        ],
       [0.08679814, 0.35647668, 0.21452212, ..., 0.31231488, 0.68444323,
        0.        ],
       [0.02309707, 0.42797927, 0.22763328, ..., 0.46965695, 0.5070641 ,
        0.        ]])

predictions for each 

In [103]:
songsom = SOM(m=3, n=6, dim=42)#recluster the values 
#songsom.fit(df_clust44)


In [104]:
songsom.fit(df_clust0)
songsom.fit(df_clust1)
songsom.fit(df_clust2)
songsom.fit(df_clust3)
songsom.fit(df_clust4)
songsom.fit(df_clust5)
songsom.fit(df_clust6)
songsom.fit(df_clust7)
songsom.fit(df_clust8)
songsom.fit(df_clust9)
songsom.fit(df_clust10)
songsom.fit(df_clust11)
songsom.fit(df_clust12)
songsom.fit(df_clust13)
songsom.fit(df_clust14)
songsom.fit(df_clust15)
songsom.fit(df_clust16)
songsom.fit(df_clust17)
songsom.fit(df_clust18)
songsom.fit(df_clust19)
songsom.fit(df_clust20)
songsom.fit(df_clust21)
songsom.fit(df_clust22)
songsom.fit(df_clust23)
songsom.fit(df_clust24)
songsom.fit(df_clust25)
songsom.fit(df_clust26)
songsom.fit(df_clust27)
songsom.fit(df_clust28)
songsom.fit(df_clust29)
songsom.fit(df_clust30)
songsom.fit(df_clust31)
songsom.fit(df_clust32)
songsom.fit(df_clust33)
songsom.fit(df_clust34)
songsom.fit(df_clust35)
songsom.fit(df_clust36)
songsom.fit(df_clust37)
songsom.fit(df_clust38)
songsom.fit(df_clust39)
songsom.fit(df_clust40)
songsom.fit(df_clust41)
songsom.fit(df_clust42)
songsom.fit(df_clust43)
#songsom.fit(df_clust44)

In [105]:
predictions = songsom.predict(df_clust0)
df_clust0_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust1)
df_clust1_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust2)
df_clust2_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust3)
df_clust3_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust4)
df_clust4_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust5)
df_clust5_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust6)
df_clust6_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust7)
df_clust7_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust8)
df_clust8_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust9)
df_clust9_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust10)
df_clust10_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust11)
df_clust11_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust12)
df_clust12_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust13)
df_clust13_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust14)
df_clust14_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust15)
df_clust15_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust16)
df_clust16_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust17)
df_clust17_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust18)
df_clust18_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust19)
df_clust19_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust20)
df_clust20_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust21)
df_clust21_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust22)
df_clust22_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust23)
df_clust23_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust24)
df_clust24_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust25)
df_clust25_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust26)
df_clust26_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust27)
df_clust27_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust28)
df_clust28_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust29)
df_clust29_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust30)
df_clust30_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust31)
df_clust31_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust32)
df_clust32_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust33)
df_clust33_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust34)
df_clust34_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust35)
df_clust35_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust36)
df_clust36_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust37)
df_clust37_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust38)
df_clust38_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust39)
df_clust39_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust40)
df_clust40_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust41)
df_clust41_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust42)
df_clust42_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust43)
df_clust43_big.insert(48, "clusterLabelsSecond", predictions, True)


In [106]:
df_clust43_big

,TrackID,TrackName,Album,PreviewURL,ArtistName,Genre,Duration,Danceability,Energy,Key,...,i,j,k,l,m,n,o,p,clusterLabels,clusterLabelsSecond
204,0yt3XJZ85Wmv0cMSG9uvqM,﻿sonate Nr. 13 in B-dur Kv 333: Andante cantabile,Mozart the Double-Faced,https://p.scdn.co/mp3-preview/7dfe8eb3c465557b...,Wolfgang Amadeus Mozart,classical,645000,0.446,0.02340,3,...,0.034588,-0.006232,0.070832,-0.005877,0.082328,-0.015219,-0.007897,-0.041416,43,17
407,6ZId2mrrWeZXSiWNxZgQMN,Daisuke & Rie,映画「ある男」 (オリジナル・サウンドトラック),https://p.scdn.co/mp3-preview/ae59a1cc8f0e3db7...,Cicada,taiwan instrumental,46613,0.583,0.00369,0,...,0.007480,-0.014134,0.073106,-0.004094,0.092428,-0.017552,0.008019,-0.044898,43,15
475,0cQlNLdaEqwSOCfTXvWpYO,Memory Express,Someday is Today,https://p.scdn.co/mp3-preview/861b84494895eb1f...,Living Hour,garage psych,282776,0.512,0.27700,2,...,0.087947,-0.002290,0.155661,0.099995,-0.010503,0.039873,0.058624,-0.040123,43,12
479,22DwH9XRoquQzDz7CSN37a,Living Hope,"Two Hours of Worship Piano, Vol. 2",https://p.scdn.co/mp3-preview/43be958ef49f4e36...,Dan Musselman,instrumental worship,295000,0.647,0.17300,2,...,0.011118,0.044399,0.075691,0.007832,0.120243,-0.067122,0.008643,-0.026040,43,10
507,5pYIpCz1qeNZhgnu7aF06F,Living In The After Hours,Pride and Joy,https://p.scdn.co/mp3-preview/3edaf49ceb42782d...,DJ Glen,brazilian edm,423946,0.832,0.50800,0,...,0.247451,0.014685,0.090870,0.005524,0.135115,0.092046,-0.018561,0.058463,43,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70618,1RGcZS0CBbbtUiLqo5J8Xc,Guided Meditation for Sublime Sleep,Guided Meditation for Sublime Sleep,https://p.scdn.co/mp3-preview/0d12f9978c56929e...,Michael Sealey,guided meditation,3576673,0.243,0.00147,0,...,-0.024620,0.026857,0.060734,-0.005588,0.099879,-0.066742,0.017196,-0.078018,43,14
70926,4l821P3mSoj52tjxLl1w11,Jazz Excellence,High Life,https://p.scdn.co/mp3-preview/c68af0c246dc992d...,Five Star Jazz,dinner jazz,77949,0.899,0.17400,0,...,-0.009552,-0.037966,0.059216,0.074910,0.098215,-0.056251,0.023698,-0.057857,43,2
70937,5WA1aGYg0uREeKrYxvAKet,Amongst The Stars,57 Yogas Welcoming Aura,https://p.scdn.co/mp3-preview/85cbc305d024d9d5...,Shakuhachi Sakano,meditation,100000,0.208,0.01170,0,...,-0.022233,-0.007850,0.081010,-0.015867,0.093741,0.027127,0.036858,-0.054947,43,15
71027,1dTZKsAz446ir15M7glV0y,My Foolish Heart,Ballad Essentials,https://p.scdn.co/mp3-preview/91c978f20e883203...,Scott Hamilton,jazz saxophone,325973,0.344,0.18000,0,...,0.058321,-0.005109,0.062482,0.045028,0.122113,0.039639,-0.065298,0.006308,43,6


In [107]:
#df_clust44_big.insert(34, "clusterLabelsSecond", predictions, True)

In [108]:
#df_clust44_big

concat all the files

In [109]:
Final_database = pd.concat([df_clust0_big, df_clust1_big,df_clust2_big,df_clust3_big,df_clust4_big,df_clust5_big,df_clust6_big,df_clust7_big,df_clust8_big,df_clust9_big,df_clust10_big,df_clust11_big,df_clust12_big,df_clust13_big,df_clust14_big,df_clust15_big,df_clust16_big,df_clust17_big,df_clust18_big,df_clust19_big,df_clust20_big,df_clust21_big,df_clust22_big,df_clust23_big,df_clust24_big,df_clust25_big,df_clust26_big,df_clust27_big,df_clust28_big,df_clust29_big,df_clust30_big,df_clust31_big,df_clust32_big,df_clust33_big,df_clust34_big,df_clust35_big,df_clust36_big,df_clust37_big,df_clust38_big,df_clust39_big,df_clust40_big,df_clust41_big,df_clust42_big,df_clust42_big,df_clust43_big]) #this is df_better_data with encoded values 
Final_database#concatonates all of the clusters into a large csv with all data to be putputted

,TrackID,TrackName,Album,PreviewURL,ArtistName,Genre,Duration,Danceability,Energy,Key,...,i,j,k,l,m,n,o,p,clusterLabels,clusterLabelsSecond
3,0ackjn8zBcLQnkeRPeOCVj,Danger,Danger,https://p.scdn.co/mp3-preview/ab856e3e92deddc1...,Cooliecut,dark trap,96052,0.720,0.56900,5,...,0.076030,-0.188622,0.159737,0.186299,0.054227,-0.214219,-0.092772,0.023308,0,12
42,5lmZInomKzuoqAqVil2aMv,"Fly, Play, Travel, Play, Fly",1 Trait World Tour,https://p.scdn.co/mp3-preview/cc0cefe8db8283c9...,1 Trait Danger,comedy rap,346791,0.902,0.60500,2,...,-0.040653,-0.096505,0.034062,0.017521,0.072396,-0.078372,-0.042377,-0.031432,0,2
380,1q9jq5X5vwmewjOa2mHtQ1,Same Yung Nigga (feat. Playboi Carti),Drip or Drown 2,https://p.scdn.co/mp3-preview/7305fbd48b7933d9...,Gunna,atl hip hop,180934,0.852,0.56800,1,...,-0.173395,0.181681,-0.047780,0.071072,0.163416,0.040743,0.001896,-0.167201,0,6
382,5QqyRUZeBE04yJxsD1OC0I,Summer Bummer (feat. A$AP Rocky & Playboi Carti),Lust For Life,https://p.scdn.co/mp3-preview/aeb292c9f842e92d...,Lana Del Rey,art pop,260672,0.760,0.57400,1,...,-0.129663,0.200700,0.177692,-0.051804,0.172673,-0.031845,0.241177,-0.082633,0,6
383,15JRvf02KKGHKgC31jrpuh,Meh,Whole Lotta Red,https://p.scdn.co/mp3-preview/f2a254ebaa780113...,Playboi Carti,atl hip hop,118500,0.764,0.51200,1,...,0.124100,0.010086,0.104090,-0.040282,0.058643,0.195508,0.222602,-0.166455,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70618,1RGcZS0CBbbtUiLqo5J8Xc,Guided Meditation for Sublime Sleep,Guided Meditation for Sublime Sleep,https://p.scdn.co/mp3-preview/0d12f9978c56929e...,Michael Sealey,guided meditation,3576673,0.243,0.00147,0,...,-0.024620,0.026857,0.060734,-0.005588,0.099879,-0.066742,0.017196,-0.078018,43,14
70926,4l821P3mSoj52tjxLl1w11,Jazz Excellence,High Life,https://p.scdn.co/mp3-preview/c68af0c246dc992d...,Five Star Jazz,dinner jazz,77949,0.899,0.17400,0,...,-0.009552,-0.037966,0.059216,0.074910,0.098215,-0.056251,0.023698,-0.057857,43,2
70937,5WA1aGYg0uREeKrYxvAKet,Amongst The Stars,57 Yogas Welcoming Aura,https://p.scdn.co/mp3-preview/85cbc305d024d9d5...,Shakuhachi Sakano,meditation,100000,0.208,0.01170,0,...,-0.022233,-0.007850,0.081010,-0.015867,0.093741,0.027127,0.036858,-0.054947,43,15
71027,1dTZKsAz446ir15M7glV0y,My Foolish Heart,Ballad Essentials,https://p.scdn.co/mp3-preview/91c978f20e883203...,Scott Hamilton,jazz saxophone,325973,0.344,0.18000,0,...,0.058321,-0.005109,0.062482,0.045028,0.122113,0.039639,-0.065298,0.006308,43,6


In [110]:
from google.colab import files


In [111]:
Final_database.to_csv('Final_database.csv', encoding = 'utf-8-sig') 
files.download('Final_database.csv') #download data to be used in next filter in the architecture 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Concatonate the clusters for easy addressing on the ones that have been double clustered 

##do this distance fucntion on a groupby call from the backend at runtime